In [26]:
#libraries
import pandas as pd
import numpy as np
from plotnine import ggplot, aes, geom_point, geom_line
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, FunctionTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [27]:
train = pd.read_csv('data/train_new.csv')

In [28]:
#check NA's
na_counts = train.isna().sum()
print(na_counts)

#probably won't use lot frontage bc its missing so much

# drop the one na for electrical
train = train.dropna(subset=['Electrical'])

train = train.drop("Lot Frontage", axis = 1)


SalePrice          0
PID                0
Lot Frontage     362
Lot Area           0
Street             0
Neighborhood       0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Roof Style         0
Heating            0
Central Air        0
Electrical         1
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
TotRms AbvGrd      0
Gr Liv Area        0
Functional         0
Screen Porch       0
Pool Area          0
Yr Sold            0
Sale Type          0
dtype: int64
SalePrice          0
PID                0
Lot Frontage     362
Lot Area           0
Street             0
Neighborhood       0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Roof Style         0
Heating            0
Central Air        0
Electrical         1
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
TotRms AbvGrd      0
Gr Liv Area        0
Functional         0
Screen Porch       0


In [29]:
#column transformer
ct = ColumnTransformer(
    [
        # log-transform Lot Area + Gr Liv Area
        ("log_transform", FunctionTransformer(np.log1p), ["Lot Area", "Gr Liv Area"]),
        
        # dummify
        ("dummify", 
         OneHotEncoder(sparse_output=False, handle_unknown="ignore"), 
         make_column_selector(dtype_include=object)),
        
        # Standardize numerical vars (not including Lot Area + Gr Liv Area)
        ("standardize", 
         StandardScaler(), 
         make_column_selector(dtype_include=np.number)),
    ],
    remainder="passthrough"
).set_output(transform="pandas")

In [30]:
#get col names
X_1 = ct.fit_transform(X)
X_1

,log_transform__Lot Area,log_transform__Gr Liv Area,dummify__Neighborhood_Blmngtn,dummify__Neighborhood_Blueste,dummify__Neighborhood_BrDale,dummify__Neighborhood_BrkSide,dummify__Neighborhood_ClearCr,dummify__Neighborhood_CollgCr,dummify__Neighborhood_Crawfor,dummify__Neighborhood_Edwards,dummify__Neighborhood_Gilbert,dummify__Neighborhood_Greens,dummify__Neighborhood_GrnHill,dummify__Neighborhood_IDOTRR,dummify__Neighborhood_Landmrk,dummify__Neighborhood_MeadowV,dummify__Neighborhood_Mitchel,dummify__Neighborhood_NAmes,dummify__Neighborhood_NPkVill,dummify__Neighborhood_NWAmes,dummify__Neighborhood_NoRidge,dummify__Neighborhood_NridgHt,dummify__Neighborhood_OldTown,dummify__Neighborhood_SWISU,dummify__Neighborhood_Sawyer,dummify__Neighborhood_SawyerW,dummify__Neighborhood_Somerst,dummify__Neighborhood_StoneBr,dummify__Neighborhood_Timber,dummify__Neighborhood_Veenker,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,dummify__House Style_1.5Fin,dummify__House Style_1.5Unf,dummify__House Style_1Story,dummify__House Style_2.5Fin,dummify__House Style_2.5Unf,dummify__House Style_2Story,dummify__House Style_SFoyer,dummify__House Style_SLvl,dummify__Central Air_N,dummify__Central Air_Y,dummify__Functional_Maj1,dummify__Functional_Maj2,dummify__Functional_Min1,dummify__Functional_Min2,dummify__Functional_Mod,dummify__Functional_Sal,dummify__Functional_Sev,dummify__Functional_Typ,standardize__PID,standardize__Lot Area,standardize__Overall Qual,standardize__Overall Cond,standardize__Year Built,standardize__Full Bath,standardize__Half Bath,standardize__Bedroom AbvGr,standardize__Gr Liv Area,standardize__Screen Porch,standardize__Yr Sold
0,9.170143,7.105786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.960824,-0.083166,0.617714,0.403970,1.149240,-1.070781,1.260820,0.169109,-0.566439,-0.282528,0.917789
1,9.594582,7.694848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.025347,0.553640,0.617714,1.307348,0.590791,0.736625,-0.738285,0.169109,1.344320,-0.282528,0.917789
2,9.573315,7.204149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.078190,0.514898,-0.081873,0.403970,-0.460408,-1.070781,-0.738285,0.169109,-0.320268,3.841770,0.917789
3,8.775395,7.284135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.977689,-0.475983,2.016888,-0.499409,1.182090,0.736625,-0.738285,-1.028836,-0.101449,-0.282528,0.917789
4,9.183483,7.226209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.977107,-0.066992,0.617714,-0.499409,1.050690,0.736625,1.260820,0.169109,-0.261655,-0.282528,0.917789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,9.214532,7.557995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.026496,-0.028501,1.317301,-0.499409,0.656491,0.736625,1.260820,0.169109,0.795318,-0.282528,-1.364730
2193,8.748464,7.145984,0.0,0.0,0.0,0.0

In [31]:
#polynomial
ct_degree = ColumnTransformer(
  [
    ("polynomial_1", PolynomialFeatures(degree = 3), ["standardize__Overall Qual"]),
    #("polynomial_2", PolynomialFeatures(degree = 2), ["standardize__bmi"]),
  ],
  remainder = "passthrough"
).set_output(transform = "pandas")

In [32]:
#interaction
ct_inter = ColumnTransformer(
  [
    ("interaction_1", PolynomialFeatures(interaction_only = True), ["standardize__Overall Cond", "dummify__Neighborhood_StoneBr"]),
    #("interaction_2", PolynomialFeatures(interaction_only = True), ["standardize__Overall Qual", "standardize__Year Built"]),
  ],
  remainder = "passthrough"
).set_output(transform = "pandas")



In [33]:
lr_pipeline_elastic = Pipeline(
  [("preprocessing", ct),
  #("polynomial", ct_degree),
  ("interaction", ct_inter),
  ("elastic_regression", ElasticNet(alpha = 1, l1_ratio = .2))]
).set_output(transform="pandas")

In [34]:
#variable selection
X = train.drop(["SalePrice", "TotRms AbvGrd", "Street", "Roof Style", "Electrical", "Heating", "Pool Area", "Sale Type"], axis = 1)
#y = train["SalePrice"]
y = np.log1p(train["SalePrice"])


In [35]:
#tune lambdas/alphas to test multiple vals
values = {"elastic_regression__alpha": [.001, .01, .1, 1, 10, 100],
          "elastic_regression__l1_ratio": [.1, .2, .3, .4, .5, .6, .7, .8, .9]}

gscv = GridSearchCV(lr_pipeline_elastic, values, cv = 5, scoring='neg_root_mean_squared_error')

gscv_fitted = gscv.fit(X, y)

params_df = pd.DataFrame(gscv_fitted.cv_results_['params'])

results_df = params_df.assign(scores=gscv_fitted.cv_results_['mean_test_score'])

results_df.sort_values(by = 'scores', ascending = False).head()

,elastic_regression__alpha,elastic_regression__l1_ratio,scores
0,0.001,0.1,-0.137137
1,0.001,0.2,-0.137192
2,0.001,0.3,-0.137410
3,0.001,0.4,-0.137685
4,0.001,0.5,-0.137984


**Testing Data**

In [36]:
#test data
test = pd.read_csv('data/test_new.csv')

**Test 1**

In [37]:
#variable selection
#X_train = train.drop(["SalePrice", "Street", "Yr Sold", "TotRms AbvGrd", "Pool Area", "Electrical"], axis = 1)
#y_train = train["SalePrice"]

#X_test = test.drop(["Street", "Yr Sold", "TotRms AbvGrd", "Pool Area", "Electrical"], axis = 1)

In [38]:
#tuned pipeline
#lr_pipeline_elastic = Pipeline(
  #[("preprocessing", ct),
  #("polynomial", ct_degree),
  #("interaction", ct_inter),
  #("elastic_regression", ElasticNet(alpha = .001, l1_ratio = .1))]
#).set_output(transform="pandas")

**Test 2**

In [39]:
#variable selection
X_train = train.drop(["SalePrice", "TotRms AbvGrd", "Street", "Roof Style", "Electrical", "Heating", "Pool Area", "Sale Type"], axis = 1)
y_train = np.log1p(train["SalePrice"])

X_test = test.drop(["TotRms AbvGrd", "Street", "Roof Style", "Electrical", "Heating", "Pool Area", "Sale Type"], axis = 1)


In [40]:
lr_pipeline_elastic = Pipeline(
  [("preprocessing", ct),
  #("polynomial", ct_degree),
  ("interaction", ct_inter),
  ("elastic_regression", ElasticNet(alpha = 0.001, l1_ratio = .1))]
).set_output(transform="pandas")

In [41]:
lr_pipeline_elastic.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('log_transform',
                                                  FunctionTransformer(func=<ufunc 'log1p'>),
                                                  ['Lot Area', 'Gr Liv Area']),
                                                 ('dummify',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x1100c9a60>),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x30a2d6210>)])),
                ('interaction',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('interaction_1',
                                                  PolynomialFeatures(interaction_only=True),
                                                  ['standardize__Overall Cond',
                                                   'dummify__Neighborhood_StoneBr'])])),
                ('elastic_regression', ElasticNet(alpha=0.001, l1_ratio=0.1))])

In [43]:
# predict y's on X_test
predictions = lr_pipeline_elastic.predict(X_test)

prediction_prices = np.exp(predictions)

final_predictions_regression = pd.DataFrame(
    {"PID": X_test['PID'],
    "SalePrice": prediction_prices}
)

#export to csv
final_predictions_regression.to_csv("regreession_test_2.csv")